In [ ]:
#-------------------------------------------------------------- CSV CLEANUP PROGRAM --------------------------------------------------------------

# The program opens the csv file of choice, recognises the anomalous data, removes it, and subsequently saves the new corrected data in a new file

#                                 !!! MAKE SURE TO REPLACE THE PATHWAY PLACEHOLDERS WITH YOUR FILE LOCATIONS !!!

#-------------------------------------------------------------------------------------------------------------------------------------------------

import pandas as pd # Allows us to read and manipulate CSV data
import csv 

def is_anomalous(cycle, start=2, period=12):
    """Allows us to manipulate and in this case cut out the anomalous data.
    It considers data to be anomalous when it is equal to the starting value (2)
    or greater going in cycles of defined size (12)"""

    return cycle >= start and (cycle - start) % period == 0

def clean_csv(input_csv, output_csv, cycle_column="cycles"):
    # Reads the input CSV file into a pandas DataFrame and further manipulates it
    df = pd.read_csv(input_csv)

    df.columns = df.columns.str.strip()

    # Removes rows where the cycle value is identified as anomalous
    df = df[~df[cycle_column].apply(is_anomalous)]

    # Writes the cleaned DataFrame to a new CSV file
    df.to_csv(
        output_csv,
        index=False, # Doesnt write row indicies to output
        sep=",", # Uses a comma to separate columns, making it excel compatible
        quoting=csv.QUOTE_MINIMAL # Only quotes fields when necessary
    )


if __name__ == "__main__":
    # Executes the program
    clean_csv(
        r"Input_Pathway", # Locates the pathway to open the original csv data file
        r"Output_Pathway" # Locates the pathway to write the new csv data file (make sure to change the title of the file if you want to keep the raw data intact)
    )


In [ ]:
#-------------------------------------------------------------- JSON CLEANUP PROGRAM --------------------------------------------------------------

# The program opens the json file of choice, recognises the anomalous data, removes it, and subsequently saves the new corrected data in a new file

#                                   !!! MAKE SURE TO REPLACE THE PATHWAY PLACEHOLDERS WITH YOUR FILE LOCATIONS !!!

#--------------------------------------------------------------------------------------------------------------------------------------------------

import json # Used to analyse and process json data

def is_anomalous(cycle, start=2, period=12):
    """Allows us to manipulate and in this case cut out the anomalous data.
    It considers data to be anomalous when it is equal to the starting value (2)
    or greater going in cycles of defined size (12)"""

    return cycle >= start and (cycle - start) % period == 0

def clean_json_ndjson(input_path, output_path, cycle_key="cycle"):
    """Opens the file and writes data into the memory"""

    with open(input_path, "r") as f:
        lines = f.readlines()

    # Determines where metadata ends using brace counting, assuming that metadata is a valid object on the top
    brace_count = 0
    metadata_lines = []
    ndjson_start_index = 0

    # Iterates through lines to find the end of the metadata block
    for i, line in enumerate(lines):
        line_strip = line.strip()
        if not line_strip:
            continue
        brace_count += line_strip.count('{')
        brace_count -= line_strip.count('}')
        metadata_lines.append(line)
        if brace_count == 0:
            ndjson_start_index = i + 1
            break

        # Combines metadata lines into a single string
    metadata_str = ''.join(metadata_lines)
    metadata = json.loads(metadata_str)

    # NDJSON records
    # Each subsequent non-empty line is expected to be a standalone JSON object
    records = []
    for line in lines[ndjson_start_index:]:
        line = line.strip()
        if not line:
            continue
        try:
            records.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Skipped invalid JSON line: {line}")

    # Removes anomalous cycles
    cleaned_records = [r for r in records if not is_anomalous(int(r[cycle_key]))]

    # Writes data in the output
    with open(output_path, "w") as f:
        f.write(json.dumps(metadata, indent=4) + "\n")
        for record in cleaned_records:
            f.write(json.dumps(record) + "\n")


if __name__ == "__main__":
    # Executes the program
    clean_json_ndjson(
        r"Input_Pathway", # Locates the pathway to open the original json data file
        r"Output_Pathway" # Locates the pathway to write the new json data file (make sure to change the title of the file if you want to keep the raw data intact)
    )


In [ ]:
#------------------------------------ SQL PROGRAM ------------------------------------

# The program reads a csv file of choice and creates an SQL library based on the data

#   !!! MAKE SURE TO REPLACE THE PATHWAY PLACEHOLDERS WITH YOUR FILE LOCATIONS !!!

#-------------------------------------------------------------------------------------

import sqlite3 # Allows SQLite database access
import pandas as pd # Allows us to read and manipulate CSV data
from pathlib import Path # Enables filesystem path handling


def csv_to_sql(csv_file, db_file, table_name="data"):
    """Converts csv into a path object"""
    csv_file = Path(csv_file)

    # Reads the csv and loads data into Panda DataFrame
    df = pd.read_csv(csv_file)

    # Standardises the column names by making them all lower case and converting all spaces and hyphens into more compatible underscores
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )

    # Cylces renamed to cycle for ease of use and improved readability (not a necessary step)
    if "cycles" in df.columns:
        df = df.rename(columns={"cycles": "cycle"})

    # Converts selected columns to numeric types, and invalid values to NaN ("Not a Number"; prevents the code from crashing when comming across an invalid value)
    numeric_colmns = ["cycle", "time", "temp", "air_quality", "humidity"]
    for colmn in numeric_colmns:
        if colmn in df.columns:
            df[colmn] = pd.to_numeric(df[colmn], errors="coerce")

    # Removes the rows with invalid values, previously saved as NaN
    df = df.dropna()

    # Adds the variable column to enable identification of the experiment
    df.insert(0, "variable", "variable_name") # REPLACE "variable_name" WITH THE EXPERIMENTAL CONDITION CHANGED

    # Connects to the SQLite database and writes the DataFrame as a table
    conn = sqlite3.connect(db_file)
    df.to_sql(table_name, conn, if_exists="replace", index=False)

    # Outputs the information about database and DataFrame table
    print(f"Table name: {table_name}")
    print(f"Rows inserted: {len(df)}")
    print("Columns in table:", df.columns.tolist())

    conn.close()


if __name__ == "__main__":
    # Executes the program

    csv_to_sql(
        r"Input_Pathway", # Locates the pathway to open the original csv data file
        r"Output_Pathway", # Locates the pathway to write the new db (database) file
        table_name="data"
    )


In [ ]:
#------------------------------- COMBINED SQL PROGRAM -------------------------------

# The program reads multiple SQL databases and combines them into one greater one
# This enables processing queries which spanning over multiple different experiments

#  !!! MAKE SURE TO REPLACE THE PATHWAY PLACEHOLDERS WITH YOUR FILE LOCATIONS !!!

#------------------------------------------------------------------------------------

import sqlite3 # Allows SQLite database access
import pandas as pd # Allows for reading SQL tables into DataFrames and writing them back
from pathlib import Path # Enables filesystem path handling

# List of SQLite databases to merge
db_files = [
    r"Input_Pathway_1",
    r"Input_Pathway_2",
    r"Input_Pathway_3",
    # add more databases here
]

# Outputs the merged database
output_db = r"Output_Pathway" # Make sure the name of the file differs from the original db file to prevent the data being overriden
table_name = "data"

# Removes output DB if it already exists clean rebuild; this is done for efficiency to ensure the table is built over each time the program is debugged
output_path = Path(output_db)
if output_path.exists():
    output_path.unlink()

# Connects to the output database
out_conn = sqlite3.connect(output_db)

# Loops through each input database and merges the data together
for i, db in enumerate(db_files):
    print(f"Merging: {db}")

    in_conn = sqlite3.connect(db)

    # Reads table into pandas
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", in_conn)

    # Appends data to the output database
    df.to_sql(
        table_name,
        out_conn,
        if_exists="append" if i > 0 else "replace",
        index=False
    )

    # Closes the connection to the input database
    in_conn.close()

# Closes the connection to the output database
out_conn.close()

print("Databases merged successfully!")


In [ ]:
#--------------------------------------------------------------------- Example for fethcing SQL queries --------------------------------------------------------------------

# The code below is an example of processing queries with the created SQL library
# The variables can be added or removed, and the conditions changed
# For the sake of the ease of use, the most important parameters are listed in the query conditions
# (time and air quality set to large numbers so the condition is always satisfied for the sake of the example, however its cleaner to remove them during actual data search)

#                                               !!! MAKE SURE TO REPLACE THE PATHWAY PLACEHOLDERS WITH YOUR FILE LOCATIONS !!!

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------


import sqlite3
import pandas as pd

# Database and table
db_file = r"Input_Pathway"
table_name = "data"

# Connects to SQLite
conn = sqlite3.connect(db_file)

# Query: selects data points that satisfy specific conditions
query = f"""
SELECT *
FROM {table_name}
WHERE CAST(time AS REAL) < 10000000
  AND CAST(temp AS REAL) < 17.5
  AND CAST(air_quality AS REAL) < 100000
  AND CAST(humidity AS REAL) < 64
"""

# Fetches results into pandas DataFrame
df = pd.read_sql_query(query, conn)

# Prints number of rows found
print(f"Found {len(df)} rows where temp < 17.5 AND humidity < 64\n") # This line can be manually altered to display the query summary

# Displays all rows without the pandas index
pd.set_option("display.max_rows", None)
print(df.to_string(index=False))

# Closes the connection
conn.close()



Found 11 rows where temp < 17.5 AND humidity < 64

        variable  cycle  time  temp  air_quality  humidity
cleaning product      1 159.0 17.20         25.0      59.0
cleaning product      3 219.0 17.31         25.0      58.0
cleaning product      4 249.0 17.36         25.0      59.0
cleaning product      5 279.0 17.39         25.0      58.0
cleaning product      6 309.0 17.38         25.0      58.0
cleaning product      7 339.0 17.40         25.0      58.0
cleaning product      8 369.0 17.44         25.0      59.0
cleaning product     10 429.0 17.47         26.0      58.0
cleaning product     11 459.0 17.47         28.0      58.0
cleaning product     12 489.0 17.49         35.0      59.0
cleaning product     13 519.0 17.49         40.0      58.0
